In [1]:
import pandas as pd
from fuzzywuzzy import process, fuzz
import sys 
sys.path.append("../") 
import src.support as sp 

In [2]:
# Convert the clean csv's into a Data Frame

df_oscar = pd.read_csv("..\data\Oscar_clean.csv")
df_goldenglobes = pd.read_csv("..\data\Golden_Globes_clean.csv")
df_sag = pd.read_csv("..\data\SAG_clean.csv")
df_cca = pd.read_csv("..\data\CCA_clean.csv")

In [3]:
# Create a tuple 'count' that contains the number of unique films in each of the specified dataframes

count = (len(df_oscar["film"].unique()), len(df_goldenglobes["film"].unique()), 
        len(df_sag["film"].unique()), len(df_cca["film"].unique()))

# Print the tuple

print(count)

(684, 722, 355, 481)


In [4]:
# Merge the 5 csv in one

merged_df = pd.concat([df_oscar, df_goldenglobes, df_sag, df_cca])

In [5]:
# Apply the function to the columns we need to standarize

merged_df["category"] = sp.standarize_title(merged_df, "category")
merged_df["film"] = sp.standarize_title(merged_df, "film")

# Strip every space contained in film

merged_df["film"] = [i.strip() for i in merged_df["film"]]

In [6]:
# Reset Index

merged_df = merged_df.reset_index()

In [7]:
# Drop the column I don´t need

merged_df.drop(columns=["Unnamed: 0"], inplace=True)
merged_df.drop(columns=["index"], inplace=True)

In [8]:
# Fill the NaN in the awards with No

merged_df["oscar"] = merged_df["oscar"].fillna("No")
merged_df["golden_globe"] = merged_df["golden_globe"].fillna("No")
merged_df["sag"] = merged_df["sag"].fillna("No")
merged_df["cca"] = merged_df["cca"].fillna("No")

In [9]:
# Checking for any duplicate

merged_df.duplicated().sum()

0

In [10]:
# Check for any null value

merged_df.isnull().sum()

year_ceremony    0
category         0
name             0
film             0
oscar            0
golden_globe     0
sag              0
cca              0
dtype: int64

In [11]:
# Replace the selected film name with the new correct one

merged_df["film"] = merged_df['film'].replace('Birdcage, The', 'The Birdcage')
merged_df["film"] = merged_df['film'].replace('Silence Of The Lambs, The', 'The Silence Of The Lambs')
merged_df["film"] = merged_df['film'].replace('Fighter, The', 'The Fighter')

In [12]:
# Take the unique values from the "film" column as an appendix

film_list = merged_df["film"].unique()

# Initialize an empty list to store the similar films

similar_films = []

# Loop through each film in the film_list 

for i, film1 in enumerate(film_list):
    year1 = merged_df[merged_df["film"] == film1]["year_ceremony"].values[0]
    for film2, year2 in [(film, merged_df[merged_df["film"] == film]["year_ceremony"].values[0]) for film in film_list[i+1:]]:
        ratio = fuzz.token_set_ratio(film1, film2)

        # Check if the similarity is greater than or equal to 80 and the year is the same

        if ratio >= 80 and year1 == year2:
            similar_films.append((film1, film2))

In [13]:
# Create a list of indexes to delete, because they are not the same movie, I check for their index first

del_values = [41, 58, 81, 95]

# Use a list comprehension to create a new list containing only the elements you want to delete
# The enumerate function returns a tuple containing the index and value of each element in the similar_films list
# The condition in the if statement filters out the elements at the indexes specified in the del_values list

similar_films1 = [film for i, film in enumerate(similar_films) if i not in del_values]

In [14]:
# Unpack the similar films into two separate lists

first_films, second_films = zip(*similar_films1)

# Replace the names so, all are the same

merged_df["film"] = merged_df['film'].replace(first_films, second_films)

In [15]:
# Convert to csv and store it in data folder

merged_df.to_csv("..\data\Cinema_Awards.csv")